In [74]:
from qiskit_ibm_runtime import QiskitRuntimeService

token = '4c635c9b66b0ab2f37ce29d7bc33edc66f815847092ba236aa79748304b2cd4bbba6c35d8a4a6adc9b579b1990cfa8066f5220cbdc29044dc9bd59f1c1e1ea1b'
instance = 'ibm-q/open/main'

service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance=instance,
        token=token
)

# Save your IBM Quantum API key in
# ~/.qiskit/qiskit-ibm.json
service.save_account(channel="ibm_quantum", 
                                  token=token,
                                  overwrite=True)

## Define the Quantum Circuit (qc)

In [75]:
from qiskit import QuantumCircuit, ClassicalRegister

def create_quantum_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    # Apply Hadamard gate to each qubit
    for i in range(num_qubits):
        qc.h(i)
    # Apply CNOT gate between each pair of qubits
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    # Add classical registers and measurements
    cr = ClassicalRegister(num_qubits)
    qc.add_register(cr)
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

# Specify the number of qubits
num_qubits = 6  # Change this value as needed
qc = create_quantum_circuit(num_qubits)

print(qc)

      ┌───┐          ┌─┐                              
 q_0: ┤ H ├──■───────┤M├──────────────────────────────
      ├───┤┌─┴─┐     └╥┘     ┌─┐                      
 q_1: ┤ H ├┤ X ├──■───╫──────┤M├──────────────────────
      ├───┤└───┘┌─┴─┐ ║      └╥┘     ┌─┐              
 q_2: ┤ H ├─────┤ X ├─╫───■───╫──────┤M├──────────────
      ├───┤     └───┘ ║ ┌─┴─┐ ║      └╥┘     ┌─┐      
 q_3: ┤ H ├───────────╫─┤ X ├─╫───■───╫──────┤M├──────
      ├───┤           ║ └───┘ ║ ┌─┴─┐ ║      └╥┘┌─┐   
 q_4: ┤ H ├───────────╫───────╫─┤ X ├─╫───■───╫─┤M├───
      ├───┤           ║       ║ └───┘ ║ ┌─┴─┐ ║ └╥┘┌─┐
 q_5: ┤ H ├───────────╫───────╫───────╫─┤ X ├─╫──╫─┤M├
      └───┘           ║       ║       ║ └───┘ ║  ║ └╥┘
c3: 6/════════════════╩═══════╩═══════╩═══════╩══╩══╩═
                      0       1       2       3  4  5 


### Print out the IBM Quantum details: instances and backends 
- to be used for Testing quality of backends at different qubit sizes

In [115]:
print(service.instances())
print(service.backends())

print(f"backend_names = {[backend.name for backend in service.backends()]}")

['ibm-q/open/main']
[<IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_brisbane')>]
backend_names = ['ibm_sherbrooke', 'ibm_kyiv', 'ibm_brisbane']


## Run the Sampler program across the QuantumCircuit(qc) and Backends defined above
- uses the service() account logged into at the top with the given token
- token should be saved in ~/.qiskit/

In [82]:
from qiskit_ibm_runtime import Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Options
options = Options()
options.optimization_level = 1
options.resilience_level = 1

# List of backend names
backend_names = ['ibm_brisbane', 'ibm_sherbrooke', 'ibm_kyiv']  # Replace with your list of backends

def run_program(backend_name, qc):
    # Get the backend
    backend = service.backend(backend_name)
    # Set up pass_manager for backend
    pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

    # Run pass_manager through transpiler
    qc_transpiled = pass_manager.run(qc)

    # Run the Sampler job
    with Session(service=service, backend=backend) as session:
        sampler = Sampler(session=session)
        job = sampler.run(circuits=[qc_transpiled])
    
    return job.job_id()

# Run the jobs on the backends and capture job_id and backend_name
jobs_backend = []
for backend_name in backend_names:
    job_id = run_program(backend_name, qc)
    jobs_backend.append((job_id, backend_name))

print(jobs_backend)

### When programs are finished, call the job_ids and compare results

In [119]:
import pandas as pd
import matplotlib.pyplot as plt
from qiskit.quantum_info import state_fidelity, Statevector

print(jobs_backend)

results = {}
for job_id, backend_name in jobs_backend:
    print(f"Getting result for {backend_name}: {job_id}")
    job = service.job(job_id)
    result = job.result()
    counts = result.get_counts()
    results[backend_name] = counts

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Display the results as a table
print(df)

# Plot the results
df.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Measurement Outcome')
plt.ylabel('Counts')
plt.title('Measurement Outcomes on Different Backends')
plt.show()

# Calculate and display fidelity between the results of different backends
ideal_backend = backend_names[0]
ideal_counts = results[ideal_backend]

fidelities = {}
for backend_name in backend_names[1:]:
    fidelity = state_fidelity(ideal_counts, results[backend_name])
    fidelities[backend_name] = fidelity

print(f"Fidelities compared to {ideal_backend}:")
for backend_name, fidelity in fidelities.items():
    print(f"{backend_name}: {fidelity}")

[('cxwq5gp4a290008xy8g0', 'ibm_brisbane'), ('cxwq5hywk6yg008j15e0', 'ibm_sherbrooke'), ('cxwq5ke4a290008xy8gg', 'ibm_kyiv')]
Getting result for ibm_brisbane: cxwq5gp4a290008xy8g0


Getting result for ibm_sherbrooke: cxwq5hywk6yg008j15e0
Getting result for ibm_kyiv: cxwq5ke4a290008xy8gg


AttributeError: 'PrimitiveResult' object has no attribute 'quasi_dists'

In [ ]:
jobs_backend = [('cxwq5gp4a290008xy8g0', 'ibm_brisbane'), ('cxwq5hywk6yg008j15e0', 'ibm_sherbrooke'), ('cxwq5ke4a290008xy8gg', 'ibm_kyiv')]
